## Giba's Property

- https://www.kaggle.com/titericz/the-property-by-giba (kernel)
- https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)

#### This kernel is just to extend giba's result in a *stupid* brute-force way
### The updated part is based on [S D's comment](https://www.kaggle.com/johnfarrell/giba-s-property-extended-result/notebook#358945)

In [1]:
%matplotlib inline
import os
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
DATA_DIR = '/Users/szkfzx/datasets/santander-value-prediction-challenge/'

hex2dec = lambda x: int(x, 16)

In [2]:
train = pd.read_csv(DATA_DIR+'train.csv')

In [3]:
cols = [
    "f190486d6","58e2e02e6","eeb9cd3aa","9fd594eec","6eef030c1","15ace8c9f",
    "fb0f5dbfe","58e056e12","20aa07010","024c577b9","d6bb78916",
    "b43a7cfd5","58232a6fb"
]
rows = np.array([2072,3493,379,2972,2367,4415,2791,3980,194,1190,3517,811,4444])-1

### The original result

In [4]:
tmp = train.loc[rows, ["ID","target"]+cols]
print('original shape', tmp.shape)
tmp

original shape (13, 15)


,ID,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb
2071,7862786dc,3513333.34,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44
3492,c95732596,160000.00,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66
378,16a02e67a,2352551.72,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,550000.00
2971,ad960f947,280000.00,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66
2366,8adafbb52,5450500.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00
4414,fd0c7cfc2,1359000.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00
2790,a36b78ff7,60000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00
3979,e42aae1b8,12000000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54
193,0b132f2c6,500000.00,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00
1189,448efbb28,1878571.42,12000000.00,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00


### Generate a subset candidates with same rows to search extendable columns

In [5]:
df_cand_col = train.loc[rows, :]
df_cand_col = df_cand_col.iloc[:, 2:]
df_cand_col

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
2071,0.0,0,0.0,0,0,0,0,0,440000.00,0.0,...,1674000.0,600000.0,3822750.0,40770000,0,0,0,0,3000000,14650000
3492,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,1674000.0,0.0,3822750.0,40770000,0,0,0,0,3000000,14650000
378,0.0,0,0.0,0,0,0,0,0,1600000.00,22000.0,...,1674000.0,0.0,3822750.0,40770000,0,0,0,0,3000000,14650000
2971,0.0,0,0.0,0,0,0,0,0,466461.54,0.0,...,1674000.0,0.0,3822750.0,40770000,0,0,790000,0,3000000,14650000
2366,0.0,0,0.0,0,0,0,0,0,3147200.00,0.0,...,1674000.0,0.0,3822750.0,40770000,0,0,0,0,3000000,14650000
4414,0.0,0,0.0,0,0,0,0,0,75000.00,26000.0,...,1674000.0,0.0,3822750.0,40770000,0,0,0,0,3000000,14650000
2790,0.0,0,0.0,0,0,0,0,0,1586888.88,0.0,...,1674000.0,0.0,3822750.0,40770000,0,0,0,0,174000,14650000
3979,0.0,0,0.0,0,0,0,0,0,1477600.00,0.0,...,1674000.0,1924000.0,3822750.0,40770000,0,0,0,0,0,14650000
193,0.0,0,0.0,0,0,0,0,0,0.00,1000000.0,...,1674000.0,90000.0,3822750.0,40770000,0,0,0,0,0,14650000
1189,0.0,0,0.0,0,0,0,0,0,310000.00,0.0,...,1674000.0,1744000.0,3822750.0,40770000,0,0,0,0,0,14650000


In [6]:
df_new = train.loc[rows, cols]

### Search in a brute-force way

In [7]:
def bf_search(df_new, df_cand):
    cnt = 0
    head_curr = df_new.values[1:, 0]
    tail_curr = df_new.values[:-1, -1]
    while True:
        for c in df_cand.columns:
            if c in df_new:
                continue
            elif np.all(
                df_cand[c].iloc[:-1].values==head_curr
            ) and len(df_cand[c].unique())>1:
                df_new.insert(0, c, df_cand[c].values)
                head_curr = df_new.values[1:, 0]
                print(c, 'found head!', 'new shape', df_new.shape)
                cnt += 1
                break
            elif np.all(
                df_cand[c].iloc[1:].values==tail_curr
            ) and len(df_cand[c].unique())>1:
                df_new[c] = df_cand[c].values
                tail_curr = df_new.values[:-1, -1]
                print(c, 'found tail!', 'new shape', df_new.shape)
                cnt += 1
                break
            else:
                continue
        if cnt==0:
            break
        else:
            cnt = 0
            continue
    return df_new

In [8]:
print('Column searching ...')
df_new = bf_search(df_new, df_cand_col)

Column searching ...
1702b5bf0 found tail! new shape (13, 14)
324921c7b found tail! new shape (13, 15)
62e59a501 found tail! new shape (13, 16)
2ec5b290f found tail! new shape (13, 17)
241f0f867 found tail! new shape (13, 18)
fb49e4212 found tail! new shape (13, 19)
66ace2992 found tail! new shape (13, 20)
f74e8f13d found tail! new shape (13, 21)
5c6487af1 found tail! new shape (13, 22)
963a49cdc found tail! new shape (13, 23)
26fc93eb7 found tail! new shape (13, 24)
1931ccfdd found tail! new shape (13, 25)
703885424 found tail! new shape (13, 26)
70feb1494 found tail! new shape (13, 27)
491b9ee45 found tail! new shape (13, 28)
23310aa6f found tail! new shape (13, 29)


In [9]:
df_new

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,...,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f
2071,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3492,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
378,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2971,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,...,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2366,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,...,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4414,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,...,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2790,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,...,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3979,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,...,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
193,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,...,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76
1189,12000000.00,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,...,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76


### Transpose the new result 
### and use the same method to search rows

In [10]:
df_new = df_new.T.copy()
df_new

,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443
f190486d6,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00,60000.00,12000000.00,500000.00,1878571.42,814800.00
58e2e02e6,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00,60000.00,12000000.00,500000.00,1878571.42
eeb9cd3aa,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00,60000.00,12000000.00,500000.00
9fd594eec,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00,60000.00,12000000.00
6eef030c1,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00,60000.00
15ace8c9f,466461.54,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00,1359000.00
fb0f5dbfe,1600000.00,466461.54,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00,5450500.00
58e056e12,0.00,1600000.00,466461.54,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72,280000.00
20aa07010,440000.00,0.00,1600000.00,466461.54,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00,2352551.72
024c577b9,3076666.66,440000.00,0.00,1600000.00,466461.54,3147200.00,75000.00,1586888.88,1477600.00,0.00,310000.00,3513333.34,160000.00


In [11]:
df_cand_row = train[df_new.index].T.copy()
df_cand_row.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
f190486d6,1866666.66,0.0,0.0,2000000.0,0.0,0.0,10000.0,0.0,3333333.34,7500000.0,...,0.0,0.0,0.0,0.0,4000000.0,0.00,242666.66,0.0,200000.0,0.0
58e2e02e6,12066666.66,2850000.0,0.0,0.0,0.0,0.0,4000.0,0.0,3925333.34,0.0,...,1600000.0,0.0,0.0,0.0,0.0,680000.00,0.00,4640000.0,0.0,0.0
eeb9cd3aa,700000.00,2225000.0,0.0,0.0,0.0,2800000.0,0.0,0.0,4000000.00,0.0,...,0.0,0.0,0.0,0.0,0.0,3191666.66,37333.34,2597000.0,0.0,0.0
9fd594eec,600000.00,1800000.0,0.0,0.0,0.0,17000000.0,30000.0,65000000.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,300000.00,0.00,0.0,1900000.0,0.0
6eef030c1,900000.00,800000.0,0.0,0.0,37662000.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,1250000.00,62000.00,14886000.0,0.0,19200000.0


In [12]:
print('Row searching ...')
df_new = bf_search(df_new, df_cand_row)

Row searching ...
1383 found head! new shape (29, 14)
3697 found tail! new shape (29, 15)
235 found tail! new shape (29, 16)
1382 found tail! new shape (29, 17)


In [13]:
df_new = df_new.T.copy()
df_new

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,...,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f
1383,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,550000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2071,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,3076666.66,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3492,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
378,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2971,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,...,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2366,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,...,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4414,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,...,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2790,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,...,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3979,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,...,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
193,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,...,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76


In [14]:
df_cand_col = train.loc[df_new.index, :]
df_cand_col = df_cand_col.iloc[:, 2:]
df_cand_col

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
1383,0.0,0,0.0,0,0,0,0,0,3076666.66,1674000.0,...,1674000.0,194000.00,3822750.0,40770000,0,0,0,0,3000000,14650000
2071,0.0,0,0.0,0,0,0,0,0,440000.00,0.0,...,1674000.0,600000.00,3822750.0,40770000,0,0,0,0,3000000,14650000
3492,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,1674000.0,0.00,3822750.0,40770000,0,0,0,0,3000000,14650000
378,0.0,0,0.0,0,0,0,0,0,1600000.00,22000.0,...,1674000.0,0.00,3822750.0,40770000,0,0,0,0,3000000,14650000
2971,0.0,0,0.0,0,0,0,0,0,466461.54,0.0,...,1674000.0,0.00,3822750.0,40770000,0,0,790000,0,3000000,14650000
2366,0.0,0,0.0,0,0,0,0,0,3147200.00,0.0,...,1674000.0,0.00,3822750.0,40770000,0,0,0,0,3000000,14650000
4414,0.0,0,0.0,0,0,0,0,0,75000.00,26000.0,...,1674000.0,0.00,3822750.0,40770000,0,0,0,0,3000000,14650000
2790,0.0,0,0.0,0,0,0,0,0,1586888.88,0.0,...,1674000.0,0.00,3822750.0,40770000,0,0,0,0,174000,14650000
3979,0.0,0,0.0,0,0,0,0,0,1477600.00,0.0,...,1674000.0,1924000.00,3822750.0,40770000,0,0,0,0,0,14650000
193,0.0,0,0.0,0,0,0,0,0,0.00,1000000.0,...,1674000.0,90000.00,3822750.0,40770000,0,0,0,0,0,14650000


In [15]:
print('Column searching (second time) ...')
df_new = bf_search(df_new, df_cand_col)

Column searching (second time) ...
e176a204a found tail! new shape (17, 30)
6619d81fc found tail! new shape (17, 31)
1db387535 found tail! new shape (17, 32)


In [16]:
print('new shape', df_new.shape)
train.loc[df_new.index, ["ID","target"]+df_new.columns.tolist()]

new shape (17, 32)


,ID,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,...,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535
1383,502cdea84,310000.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,440000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2071,7862786dc,3513333.34,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,0.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3492,c95732596,160000.00,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,1600000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
378,16a02e67a,2352551.72,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,466461.54,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2971,ad960f947,280000.00,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,3147200.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2366,8adafbb52,5450500.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,75000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4414,fd0c7cfc2,1359000.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,1586888.88,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
2790,a36b78ff7,60000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,1477600.00,...,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3979,e42aae1b8,12000000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,0.00,...,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
193,0b132f2c6,500000.00,60000.00,1359000.00,5450500.00,280000.00,2352551.72,160000.00,3513333.34,310000.00,...,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


In [17]:
print(f'Row indexes({df_new.shape[0]})\n', df_new.index.values.tolist())
print(f'Column indexes({df_new.shape[1]})\n', df_new.columns.values.tolist())

Row indexes(17)
 [1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382]
Column indexes(32)
 ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', '6619d81fc', '1db387535']


## There's a strange long tail of number ***1563411.76*** pointed out by [S D](https://www.kaggle.com/johnfarrell/giba-s-property-extended-result/comments#358945)
## Thanks [S D](https://www.kaggle.com/sdoria) for this new idea!
## Let's check it

In [18]:
for i, c in enumerate(df_new.columns):
    print(
        'No.', i, 'Column Name', c, 
        'subset count',
        (df_new[c].values==1563411.76).sum(), 
        'train count',
        (train[c].values==1563411.76).sum()
    )

No. 0 Column Name f190486d6 subset count 0 train count 0
No. 1 Column Name 58e2e02e6 subset count 0 train count 1
No. 2 Column Name eeb9cd3aa subset count 0 train count 2
No. 3 Column Name 9fd594eec subset count 0 train count 3
No. 4 Column Name 6eef030c1 subset count 0 train count 4
No. 5 Column Name 15ace8c9f subset count 0 train count 5
No. 6 Column Name fb0f5dbfe subset count 0 train count 6
No. 7 Column Name 58e056e12 subset count 0 train count 7
No. 8 Column Name 20aa07010 subset count 0 train count 7
No. 9 Column Name 024c577b9 subset count 0 train count 8
No. 10 Column Name d6bb78916 subset count 0 train count 9
No. 11 Column Name b43a7cfd5 subset count 0 train count 10
No. 12 Column Name 58232a6fb subset count 0 train count 11
No. 13 Column Name 1702b5bf0 subset count 0 train count 12
No. 14 Column Name 324921c7b subset count 0 train count 13
No. 15 Column Name 62e59a501 subset count 0 train count 13
No. 16 Column Name 2ec5b290f subset count 1 train count 14
No. 17 Column Name

In [19]:
res_cnt = dict((c, (train[c].values==1563411.76).sum()) for c in train.columns[2:])
res_cnt = pd.DataFrame.from_dict(res_cnt, orient='index', columns=['strange_number_cnt'])
res_cnt = res_cnt.sort_values('strange_number_cnt', 0, False)
res_cnt.head(50).T

,0ff32eb98,c5a231d81,c47340d97,adb64ff71,190db8488,0572565c2,91f701ba2,fc99f9426,1db387535,6619d81fc,...,7f0d863ba,b22eb2036,e70581bed,adbe5fd20,c30399758,48df886f9,ea3f3029c,b88e5de84,1aa44bdf8,8e978ee65
strange_number_cnt,36,35,34,33,32,31,30,30,29,28,...,0,0,0,0,0,0,0,0,0,0


In [20]:
res_cnt.head(10)

,strange_number_cnt
0ff32eb98,36
c5a231d81,35
c47340d97,34
adb64ff71,33
190db8488,32
0572565c2,31
91f701ba2,30
fc99f9426,30
1db387535,29
6619d81fc,28


## Sad... there are *2* columns with the same count 30!
- 91f701ba2
- fc99f9426

## What about the row-wise?

In [21]:
for i, c in enumerate(df_new.T.columns):
    print(
        'No.', i, 'Row Name', c, 
        'subset count',
        (df_new.T[c].values==1563411.76).sum(), 
        'train count',
        (train.T[c].values==1563411.76).sum()
    )

No. 0 Row Name 1383 subset count 16 train count 24
No. 1 Row Name 2071 subset count 15 train count 23
No. 2 Row Name 3492 subset count 14 train count 22
No. 3 Row Name 378 subset count 13 train count 21
No. 4 Row Name 2971 subset count 12 train count 20
No. 5 Row Name 2366 subset count 11 train count 19
No. 6 Row Name 4414 subset count 10 train count 18
No. 7 Row Name 2790 subset count 9 train count 17
No. 8 Row Name 3979 subset count 8 train count 16
No. 9 Row Name 193 subset count 7 train count 15
No. 10 Row Name 1189 subset count 6 train count 14
No. 11 Row Name 3516 subset count 5 train count 13
No. 12 Row Name 810 subset count 4 train count 12
No. 13 Row Name 4443 subset count 3 train count 11
No. 14 Row Name 3697 subset count 2 train count 10
No. 15 Row Name 235 subset count 1 train count 9
No. 16 Row Name 1382 subset count 0 train count 8


In [22]:
tmp = train.iloc[:, 2:].values
res_t_cnt = dict((idx, (tmp[i, :]==1563411.76).sum()) for i,idx in enumerate(train.index))
res_t_cnt = pd.DataFrame.from_dict(res_t_cnt, orient='index', columns=['strange_number_cnt'])
res_t_cnt = res_t_cnt.sort_values('strange_number_cnt', 0, False)
res_t_cnt.head(50).T

,1757,3809,511,3798,625,3303,4095,1283,4209,1696,...,2975,2989,2983,2976,2985,2973,2977,2978,2981,2987
strange_number_cnt,39,38,37,36,35,34,33,31,30,29,...,0,0,0,0,0,0,0,0,0,0


## Hmm, seems no same values. OK, we start from rows!

In [23]:
head_row_indexes = res_t_cnt[res_t_cnt['strange_number_cnt']>24].index.tolist()
head_row_indexes

[1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245]

In [24]:
mask = res_t_cnt['strange_number_cnt']>0 
mask&=res_t_cnt['strange_number_cnt']<8
tail_row_indexes = res_t_cnt.loc[mask].index.tolist()
tail_row_indexes

[4384, 3418, 4396, 921, 3176, 650]

In [25]:
pd.concat([
    train.loc[head_row_indexes, ['target']+df_new.columns.tolist()], 
    train.loc[df_new.index, ['target']+df_new.columns.tolist()],
    train.loc[tail_row_indexes, ['target']+df_new.columns.tolist()], 
])

,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,...,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535
1757,115636.36,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,834800.00,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,296444.44,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,247166.66,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,550000.00,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,3076666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,440000.00,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,1600000.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4209,466461.54,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1696,3147200.00,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


## Good! This is our new df_new and we got to search once more!

In [26]:
df_new = pd.concat([
    train.loc[head_row_indexes, df_new.columns.tolist()], 
    train.loc[df_new.index, df_new.columns.tolist()],
    train.loc[tail_row_indexes, df_new.columns.tolist()], 
])

In [27]:
def row_bf_search(df_new):
    df_new = df_new.T.copy()
    df_cand_row = train[df_new.index].T.copy()
    print('Row searching ...')
    df_new = bf_search(df_new, df_cand_row)
    df_new = df_new.T.copy()
    return df_new
def column_bf_search(df_new):
    df_cand_col = train.loc[df_new.index, :]
    df_cand_col = df_cand_col.iloc[:, 2:]
    print('Column searching ...')
    df_new = bf_search(df_new, df_cand_col)
    return df_new

In [28]:
df_new = column_bf_search(df_new)

Column searching ...


In [29]:
df_new = row_bf_search(df_new)

Row searching ...


## No good... But wait! Let's check it!
### Here, we still can't identify the true order of the columns with the same count 30
### Just for example take a look at 'fc99f9426','91f701ba2' at first
### We add 'fc99f9426','91f701ba2' and the first 6 columns with count 31~36

In [30]:
res_cnt[:10]

,strange_number_cnt
0ff32eb98,36
c5a231d81,35
c47340d97,34
adb64ff71,33
190db8488,32
0572565c2,31
91f701ba2,30
fc99f9426,30
1db387535,29
6619d81fc,28


In [31]:
train.loc[df_new.index, ['target']+df_new.columns.tolist()+['fc99f9426','91f701ba2'] + res_cnt.index.values[:6].tolist()[::-1]]

,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,...,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
1757,115636.36,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,834800.00,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,296444.44,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,247166.66,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,550000.00,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,3076666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,440000.00,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,1600000.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4209,466461.54,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1696,3147200.00,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


## Hmm, seems something is wrong
### at column: ['1db387535', 'fc99f9426','91f701ba2']
### from ~last 10 rows

In [32]:
train.loc[df_new.index[-10:], df_new.columns.tolist()[-2:]+['fc99f9426', '91f701ba2'] + res_cnt.index.values[:6].tolist()[::-1]]

,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
4443,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3697,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
235,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1382,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4384,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3418,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4396,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76
921,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76
3176,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76
650,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76


### Check the table above
### Seems 540000.00 disappeared from column '1db387535'
###      And 1015000.00	 disappeared from column 'fc99f9426'
### Thus the time series is becoming not *serious* from here...
### Another question, what about the order of 'fc99f9426','91f701ba2'? Which's first?

In [33]:
train.loc[df_new.index[:8], ['f190486d6', '58e2e02e6']].T

,1757,3809,511,3798,625,3303,4095,1283
f190486d6,1015000.00,540000.0,115636.36,834800.00,296444.44,247166.66,550000.00,440000.00
58e2e02e6,1563411.76,1015000.0,540000.00,115636.36,834800.00,296444.44,247166.66,3076666.66


In [34]:
train.loc[df_new.index[-2:], df_new.columns.tolist()[-2:]+['fc99f9426', '91f701ba2']+res_cnt.index.values[:6].tolist()[::-1]]

,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
3176,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.0,1563411.76,1563411.76
650,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.0,1015000.00,1563411.76


### From row values (in reverse order), it seems 'fc99f9426', '91f701ba2' is consistent with other columns 
### Thus, in our limited analysis, the new df_new is

In [35]:
df_new_new = train.loc[df_new.index, df_new.columns.tolist()+['fc99f9426','91f701ba2'] + res_cnt.index.values[:6].tolist()[::-1]]
df_new_new.shape

(36, 40)

In [36]:
print(f'Row indexes({df_new_new.shape[0]})\n', df_new_new.index.values.tolist())
print(f'Column indexes({df_new_new.shape[1]})\n', df_new_new.columns.values.tolist())

Row indexes(36)
 [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]
Column indexes(40)
 ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']


In [37]:
train.loc[df_new_new.index, ['ID', 'target']+df_new_new.columns.tolist()]

,ID,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,...,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
1757,6726fff18,115636.36,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,d94655f86,834800.00,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,1df3ca92e,296444.44,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,d8e48b069,247166.66,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,24204cd10,550000.00,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,bf6c2d1ef,3076666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,eacc7ab9e,440000.00,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.00,1563411.76,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,4a5425356,1600000.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4209,f0a57697d,466461.54,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,115636.36,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1696,625e88875,3147200.00,1600000.00,0.00,440000.00,3076666.66,550000.00,247166.66,296444.44,834800.00,...,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


### For the final result consist many uncertains and hypothesis
### Be careful of using this !!!
### Any comments and advices are welcome !!!
### Please point out if there's something wrong!!!